# 多进程

## 1. 多进程的陷阱

### `fork()`

什么时候会调用 `fork` 函数呢？
-  需要并发执行
- 调用系统上额外的函数


下面这段代码有什么问题？

In [ ]:
int createChildAndSayHey() {
    pid_t pid = fork();

    if (pid == 0) {
        sayHey();
        return 0;
    }

    waitpid(pid);
}

- 从 `main` 函数返回，进程退出
- 从别的函数返回，返回他的调用者
- 子进程得到父进程的所有调用栈
- 子进程并没有退出 —— 它正在关闭并执行可能为父进程准备的代码

## 2. 混淆进程和线程

- 与进程不同，线程会共享虚拟地址空间
- `malloc` 是线程安全的
    - 使用 `lock` 确保两个线程不能够并发调用 `malloc` -- 确保一个线程不能够中断另一个线程的堆区间
- 如果父进程调用 `fork` 时，有多个活动的线程，没关系，只有一个线程将存在于子进程中（刚刚调用 fork 的那个）。
    - fork 之后，只有一个线程在子进程中运行
- 在调用 `fork` 时，孩子会得到所有父进程的内存的副本
    - 包括堆栈等信息
    - 锁的状态

> 你可能确保你的代码不存在任何线程，但是
> - 你能确保你调用的库中没有使用任何线程？
> - 你能确保没有任何后台线程？

- 实际上，如果子进程要在克隆的内存空间中做任何有意义的工作，不要将多进程与线程混合使用
    - 在实际中，很难保证在你调用的过程中没有任何线程
- 最大的危险往往伴随着 `fork()`
- 如果使用多处理并发执行，把你想并发运行的代码放在**一个单独的可执行**中
    - 使用 `exec` 调用将会「重置」虚拟内存空间



## 3. 为什么要分离 `fork` 和 `exec` ?

- Linux: 定制化 & 简单性
    - 重新连接文件描述符？更改一些环境变量？屏蔽信号？将进程固定到特定的 CPU 核心（缓存优化）？
    - 也许可以更加灵活？

## 4. 常见的多进程策略

- 如果有需要，`fork` & `exec` 的灵活性就在那
- 定义/使用一个更高级的抽象去处理常见错误
    - ex: 'subprocess'
    - 类似于 Windows 方法，但操作系统无需涵盖所有可能的有效用例
- 大多数这些抽象允许您重定向标准输入/输出并提供您希望在 `fork` 之后和 `exec` 之前执行的功能。

## 5. `pipe()`: 将会发生什么错误呢？

__pipe 的问题__
- 文件描述符泄漏
- 在错误值上面调用 `close()`
    - 比如：
        ```c
        if (close(fds[1] == -1)) {
            printf("Error closing!");
        }
        ```
- 提前使用了 pipe
- 在 `close` 之后使用

__潜在的解决方案__
- 定义一个 pipe 类型，而不是数字
- 写到一个标准输入的 pipe:
    ```rs
    let mut child = Command::new("cat")
                .stdin(Stdio::piped())
                .stdout(Stdio::piped())
                .spawn()?;
    child.stdin.as_mut().unwrap().write_all(b"Hello, world!\n")?;
    let output = child.wait_with_output()?;
    ```